In [1]:
import numpy as np

import sys
sys.path.insert(0, '../../src/simulator')     # import path/to/synthpy

import config
config.jax_init()

import beam as beam_initialiser
import domain as d
import propagator_improving as p

import importlib
importlib.reload(beam_initialiser)
importlib.reload(d)
importlib.reload(p)


Initialising jax...

jax:    0.6.2
jaxlib: 0.6.2
numpy:  1.26.4
python: 3.12.5 | packaged by Anaconda, Inc. | (main, Sep 12 2024, 18:27:27) [GCC 11.2.0]
device info: cpu-16, 16 local devices"
process_count: 1
platform: uname_result(system='Linux', node='computer', release='5.15.179-1-MANJARO', version='#1 SMP PREEMPT Thu Mar 13 13:45:22 UTC 2025', machine='x86_64')

Default jax backend: cpu
Available devices: [CpuDevice(id=0), CpuDevice(id=1), CpuDevice(id=2), CpuDevice(id=3), CpuDevice(id=4), CpuDevice(id=5), CpuDevice(id=6), CpuDevice(id=7), CpuDevice(id=8), CpuDevice(id=9), CpuDevice(id=10), CpuDevice(id=11), CpuDevice(id=12), CpuDevice(id=13), CpuDevice(id=14), CpuDevice(id=15)]


<module 'propagator_improving' from '/home/administrator/Work/UROP_ICL_Internship/synthPy/evaluation/MWE/../../src/simulator/propagator_improving.py'>

In [2]:
extent_x = 5e-3
extent_y = 5e-3
extent_z = 10e-3

n_cells = 128

probing_extent = extent_z
probing_direction = 'z'

lengths = 2 * np.array([extent_x, extent_y, extent_z])

domain = d.ScalarDomain(lengths, n_cells, ne_type = "test_exponential_cos")

Predicted size in memory of domain: 8.0 MB
Free memory: 7.55401611328125 GB

Generating test exponential decay periodic -e field.

jax:    0.6.2
jaxlib: 0.6.2
numpy:  1.26.4
python: 3.12.5 | packaged by Anaconda, Inc. | (main, Sep 12 2024, 18:27:27) [GCC 11.2.0]
device info: cpu-16, 16 local devices"
process_count: 1
platform: uname_result(system='Linux', node='computer', release='5.15.179-1-MANJARO', version='#1 SMP PREEMPT Thu Mar 13 13:45:22 UTC 2025', machine='x86_64')


In [3]:
lwl = 1064e-9 #define laser wavelength

Np = 16
divergence = 5e-5
beam_size = extent_x
ne_extent = probing_extent

beam_definition = beam_initialiser.Beam(
    Np,
    beam_size,
    divergence,
    ne_extent,
    probing_direction = probing_direction,
    wavelength = lwl,
    beam_type = "circular")

#ne_nc, x, y, z, x_n, y_n, z_n, extent
# unpack into 2 as jonesvector return None for Jf when not desired
rf, duration = p.solve(
    beam_definition.s0,
    (domain.x, domain.y, domain.z),
    (domain.x_n, domain.y_n, domain.z_n),
    domain.lengths[['x', 'y', 'z'].index(probing_direction)] / 2,
    None,
    *p.calc_dndr(domain)[1:]
)

print("\nCompleted ray trace in", np.round(duration, 3), "seconds.")


Size in memory of initial rays: 712 B

Running device: cpu, with: 16 cores.
NamedSharding(mesh=Mesh('rows': 16, axis_types=(Auto,)), spec=PartitionSpec('rows', None), memory_kind=unpinned_host)

jax compilation of solver took: 0.00034117698669433594 seconds

Parallelised output has resulting 3D matrix of form: [batch_count, 2, 9]: (16, 2, 9)
	2 to account for the start and end results
	9 containing the 3 position and velocity components, amplitude, phase and polarisation
	If batch_count is lower than expected, this is likely due to jax's forced integer batch sharding when parallelising over cpu cores.

We slice the end result and transpose into the form: (9, 16) to work with later code.

Completed ray trace in 3.631 seconds.
